In [ ]:
import torch

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.cnn_layers = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.BatchNorm2d(12),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
            
            torch.nn.Conv2d(in_channels = 12, out_channels = 24, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.BatchNorm2d(24),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
            
            torch.nn.Conv2d(in_channels = 24, out_channels = 48, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.BatchNorm2d(48),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        
        self.linear_layers = torch.nn.Sequential(
            torch.nn.Linear(4*4*48, 10)
        )

    def forward(self,x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1) # Flattening the input
        x = self.linear_layers(x)
        return x

In [ ]:
model = Net()
model.load_state_dict(torch.load('data/cifar_net.pth'))
model = model.cuda()

In [ ]:
import pickle
import numpy as np
from sklearn.metrics import accuracy_score

def unpickle(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
        X = batch[b'data']/255 - 0.5
        X = X.reshape(X.shape[0], 3, 32, 32)#.transpose(0, 2, 3, 1)
        y = batch[b'labels']
    return X,y
X_test,y_test = unpickle('data/cifar-10-batches-py/test_batch')

In [ ]:
with torch.no_grad():
    _,predicted = torch.max(model( torch.from_numpy(X_test).float().cuda() ),1)
    predicted = predicted.cpu().numpy()
    print('Test Accuracy : ', round(accuracy_score(y_true = y_test, y_pred = predicted) * 100,2))
torch.cuda.empty_cache()